In [0]:
# INCLUDE_HEADER_TRUE
# INCLUDE_FOOTER_TRUE

# Using Databricks Utilities and CLI

In this lab you will learn how to:
* Use Databricks utilities (**dbutils**) to perform various tasks from within your notebooks
* Install, configure and use the Databricks CLI to remotely administer Databricks

##Prerequisites

If you would like to follow along with this lab, you will need access to a cluster with *Single user* access mode. The *Shared* access mode does not support the operations required by this lab.

## Databricks Utilities

**dbutils** is a collection of utility functions that empowers you to do more within your notebooks, making it easy to perform powerful combinations of tasks. You can use the utilities to work with object storage efficiently, chain and parametrize notebooks, and to work with secrets. dbutils are only available in the context of a notebook and have Python, R and Scala bindings.

In this lab, we will explore usage using Python. Refer to the <a href="https://docs.databricks.com/dev-tools/databricks-utils.html" target="_blank">documentation</a> for other languages.

For starters, let's invoke the online help to see available commands.

In [0]:
dbutils.help()

### Parametrizing notebooks with widgets

Let's look at an example usage of dbutils: creating input widgets that allow you to add parameters to your notebooks and dashboards. Let's try a simple example that creates a text input field named *token*.

In [0]:
dbutils.widgets.text(name='token', defaultValue='')

#### Widgets in Python applications

As of DBR 11.0, Databricks recommends using **ipywidgets** for interactive widgets in Python applications. **ipywidgets** provides a broader suite of widgets with a richer programming interface.

In this lab, we'll stick to Databricks widgets, but for the sake of illustration, the code below exemplifies usage that is roughly equivalent to what we're doing here:
```
import ipywidgets as w
token_widget = w.Text(description='Token:')
token_widget
```
The input value can be retrieved as follows:
```
token_widget.value
```

### Accessing DBFS

Let's look at another example usage of dbutils: accessing and managing DBFS (Databricks File System). DBFS is a distributed file system mounted into a Databricks workspace and available on all Databricks clusters associated with that workspace. DBFS is an abstraction on top of scalable object storage that provides an optimized interface that maps to native cloud storage API calls. It's peristent, unlike the ephemeral file systems that clusters use, and provides a convenient location for exchanging information between the workspace and running clusters.

Let's access the online help for the **`fs`** command.

In [0]:
dbutils.fs.help()

#### Reading from DBFS

Now let's invoke the **`fs`** command to display the contents of the topmost folder in DBFS. Wrapping this into a call to **`display()`** nicely formats it as a table.

In [0]:
display(dbutils.fs.ls("/"))

#### Creating a file in DBFS

Let's create a file in DBFS, */tmp/token* (overwriting, if needed) that contains the value specified in the widget we created earlier.

In [0]:
dbutils.fs.put('/tmp/token', dbutils.widgets.get('token'), True)

Before proceeding, create a personal access token and paste its value into the *token* widget.
1. Go to <a href="#setting/account" target="_blank">User Settings</a> (which is also accessible from the left sidebar by selecting **Settings > User Settings**).
1. Select the **Access tokens** tab.
1. Click **Generate new token**.
    1. Specify a **Comment** such as *CLI Test*. Choose a short value for **Lifetime**; for the purpose of this lab, one or two days is sufficient.
    1. Click **Generate**.
    1. Copy the resulting token to the clipboard and click **Done**.
1. Paste the generated token into the *token* widget and proceed.

Default widget behavior will automatically trigger the execution of any cell that references the widget's value; thus the */tmp/token* file will automatically be rewritten after you paste the token.

## Databricks command-line interface

The Databricks command-line interface (CLI) enables you to automate tasks through an easy-to-use interface to the Databricks platform. Built on top of the Databricks REST APIs, Databricks CLI is organized into command groups allowing you to perform a variety of tasks.

Users typically install and run the client in their own environment, where they can invoke commands manually or integrate the CLI with an automation framework of some sort. The CLI is implemented in Python and can be used from any shell-based environment that has Python support (2.7.9+ or 3.7+).

In this lab, we will take advantage of the execution environment provided by the attached all-purpose cluster for the purpose of demonstrating installation and usage.

### Installing the CLI

Let 's install the CLI using **`pip`**. Please note the following peculiarites that relate to performing this on a Databricks cluster using a notebook:

* We precede the actual command with the **`%sh`** magic command to run it in a command-line shell on the cluster. In a conventional shell environment, this would not be necessary and we would simply issue the **`pip install`** command directly.
* The resulting software is installed as part of an ephemeral execution environment. You will need to reinstall and reconfigure the CLI if the cluster is restarted, or even if you simply reattach to it. In a conventional environment, the installation and configuration would be persistent.

In [0]:
%sh pip install databricks-cli

Let's perform a basic test of the installation by invoking the **`databricks`** command. Once again, we need to precede the command with the **`%sh`** magic command since we are running this in the context of a notebook cell.

In [0]:
%sh databricks --help

### Configuring the CLI

Before we can run any commands that interact with the Databricks environment, we need to configure the CLI, which sets up authentication. Let's review the options available.

In [0]:
%sh databricks configure --help

Let's configure the CLI now by specifying the URL of our workspace and a personal access token as authentication credentials.

Before running this cell, replace the string *URL* with the URL of your workspace, stripping off the path (that is, everything after the host).

For the token, we're using the file created earlier in conjunction with the **`--token-file`** option. When running this in an interactive shell environment, you would have the option using the **`--token`** option and specifying the token interactively.

In [0]:
%sh databricks configure --host URL --token-file /dbfs/tmp/token

With this simple configuration complete, you can now begin managing your Databricks environment through the CLI. Note that in this lab we are administering the current workspace, though we can administer any workspace this way with the appropriate URL and token generated from that workspace.

### Using the CLI

The CLI essentially provides an interface to the APIs that's easy to use from a command-line shell. Not all of the APIs are covered, but there's a lot you can do with the command-line interface, and we're only going to scratch the surface here. For a full discourse on all the options available, refer to the <a href="https://docs.databricks.com/dev-tools/cli/index.html" target="_blank">documentation</a>.

#### Exploring and managing the data hierarchy

Let's examine the data hierarchy beginning at the metastore level. First, let's list the metastores available.

In [0]:
%sh databricks unity-catalog metastores list

This lists information about *all* metastores defined in the Databricks account governing this workspace. The following cell retrieves information regarding the metastore currently assigned to the workspace.

In [0]:
%sh databricks unity-catalog metastores get-summary

Now let's browse the structure within the currently assigned metastore, starting with the catalogs.

In [0]:
%sh databricks unity-catalog catalogs list

Notice that *hive_metastore* and *samples* are not listed, since these are not real catalogs. They are virtual entries mapped into the catalog namespace by Unity Catalog for convenient access to the local Hive metastore and Databricks sample datasets.

Now let's list the schemas within the catalog *main*.

In [0]:
%sh databricks unity-catalog schemas list --catalog-name main

Now let's see the tables within the *default* schema of *main*.

In [0]:
%sh databricks unity-catalog tables list --catalog-name main --schema-name default

Now that we've browsed the data structures, let's create a new schema named *myschema_cli* within the *main* catalog. Note that this will fail if you don't have appropriate privileges on *main*.

In [0]:
%sh databricks unity-catalog schemas create --catalog-name main --name myschema_cli

Open the **Data** page to validate the creation of the schema.

#### Exploring and managing workspace assets

So far we've used the CLI to browse and manage data assets, but it's also possible to automate the management of workspace assets and compute resources through the CLI. As an example, let's use the CLI to create an all-purpose cluster. Let's first take a look at the options available.

In [0]:
%sh databricks clusters create --help

Let's create a JSON file in DBFS to describe the cluster configuration. The parameters specified herre are explained in the <a href="https://docs.databricks.com/api/latest/clusters.html#create" target="_blank">API documentation</a>, though you can also obtain sample JSON from the cluster creation UI as well.

Note that the configuration file is not strictly necessary; the **`--json`** option allows us to specify JSON inline, however this is cumbersome to use so we use a file instead.

In [0]:
dbutils.fs.put(
    "/tmp/cluster.json",
    """{
        "num_workers": 1,
        "cluster_name": "mycluster_cli",
        "spark_version": "11.1.x-scala2.12",
        "node_type_id": "i3.xlarge",
        "autotermination_minutes": 120,
        "data_security_mode": "USER_ISOLATION",
        "runtime_engine": "STANDARD"
    }""",
    True)

With the configuration file in place, let's create the cluster.

In [0]:
%sh databricks clusters create --json-file /dbfs/tmp/cluster.json

Note the value for *cluster_id*; we will need this shortly.

Now open the <a href="#setting/clusters" target="_blank">Compute</a> page (which is also accessible from the left sidebar) to validate the creation of the cluster.

## Cleanup

Run the following cells to remove the resources we created throughout this lab. This further illustrates how dbtuils and CLI can be used to manage resources.

First, let's clean up the DBFS files.

In [0]:
dbutils.fs.rm("/tmp/cluster.json")
dbutils.fs.rm("/tmp/token")

Let's use the CLI to delete the schema we created.

In [0]:
%sh databricks unity-catalog schemas delete --full-name main.myschema_cli

Finally, let's use the CLI to delete the cluster we created. Note that you must subsitute the text *CLUSTER_ID* with the actual cluster id, which was output from the cluster create command earlier. Let's perform that substitution, then run the cell.

In [0]:
%sh databricks clusters permanent-delete --cluster-id "CLUSTER_ID"